In [1]:
import spacy
import math
from random import randint
from tqdm import tqdm
nlp = spacy.load('ja_ginza')

In [18]:
mask_ratio = 0.5
span_length =3

In [3]:
sentences = []
for line in open('/home/chen/T5_mask_infilling/clinic_corpus.txt', 'r'):
    if line is not None:
        sentences.append(line.strip())

In [4]:
len(sentences)

11832

In [5]:
import torch
from transformers import T5Tokenizer, T5Config, T5ForConditionalGeneration

T5_PATH = '/home/chen/T5_mask_infilling/torch_model_clinic' 

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 


t5_tokenizer = T5Tokenizer.from_pretrained(T5_PATH)
t5_config = T5Config.from_pretrained(T5_PATH)
t5_mlm = T5ForConditionalGeneration.from_pretrained(T5_PATH, config=t5_config).to(DEVICE)

In [19]:
gen_list = []
input_list = []
for i in tqdm(range(len(sentences)-11800)):    
    
    doc = nlp(sentences[i])
    mask_token = math.ceil(len(doc)*mask_ratio)
    chunk_num = math.ceil(mask_token/span_length)
    total_chunk = chunk_num
    assert chunk_num != 0
    
    index_sequence = [i for i in range(len(doc)-span_length+1)]
    for i in range(len(index_sequence)):
        index_sequence[i] = [i for i in range(i,i+span_length)] 
        
    mask_index = []
    first_token_mask = False
    last_token_mask = False
    while chunk_num > 0:
        index_sequence_length = len(index_sequence)
        masked_spans = index_sequence[randint(0,index_sequence_length-1)]
        if masked_spans[0] == 0:
            first_token_mask = True
        if masked_spans[0] ==  len(doc)-span_length:
            last_token_mask = True
        mask_index.extend(masked_spans)
        index_id = index_sequence.index(masked_spans)
        del index_sequence[index_id-span_length+1:index_id+span_length]
        chunk_num = chunk_num - 1    
        
    mask_index.sort()
    
    text_sequence = [i for i in range(len(doc))]
    unmask_index = list(set(text_sequence) - set(mask_index))
    
    unmask_index.sort()
    
    input_text = [None] * len(doc)
    for i in unmask_index:
        input_text[i] = doc[i]
        
        
    re_input_text = []
    for i in range(len(input_text)-1):
        if str(input_text[i]) == 'None' and str(input_text[i+1]) == 'None':
            pass
        else:
            re_input_text.append(input_text[i])
        i+=2               

    re_input_text.append(input_text[len(input_text)-1])   
    
    sent_token_id = 0
    for i in range(len(re_input_text)):
        if str(re_input_text[i]) == 'None':
            re_input_text[i] = f'<extra_id_{sent_token_id}>'
            sent_token_id += 1  
            
    inputs = ''.join(str(e) for e in re_input_text)
    input_list.append(inputs)
    text = inputs

    encoded = t5_tokenizer(text, add_special_tokens=True, return_tensors='pt', 
                       padding = 'max_length', truncation = True, max_length = 128)
    input_ids = encoded['input_ids'].to(DEVICE)
    attention_mask = encoded['attention_mask'].to(DEVICE)
    outputs = t5_mlm.generate(input_ids = input_ids, 
                          num_beams=200, num_return_sequences=1,
                          max_length=15*total_chunk)

    for i in range(len(outputs)):
        query = t5_tokenizer.decode(outputs[i], skip_special_tokens=False, 
                                    no_repeat_ngram_size=3, early_stopping=True, clean_up_tokenization_spaces=False)

        if first_token_mask is True and last_token_mask is False:
            sent_token_id = 0
            s = str(query)
            start = '<pad>'
            end = '<extra_id_0>'
            pattern_0 = s[s.find(start)+len(start):s.rfind(end)]
            pattern_list = []
            pattern_list.append(pattern_0)
            i = 0
            while i < total_chunk-1:
                start = f'<extra_id_{sent_token_id}>'
                end = f'<extra_id_{sent_token_id+1}>'
                a = s[s.find(start)+len(start):s.rfind(end)]
                pattern_list.append(a)
                sent_token_id += 1 
                i +=1
            
            sent_token_id = 0
            for i in range(len(pattern_list)):
                a = f'<extra_id_{sent_token_id}>'
                text = text.replace(a, pattern_list[i].strip())
                sent_token_id += 1
            gen_list.append(text.strip())
            
        if first_token_mask is True and last_token_mask is True:
            sent_token_id = 0
            s = str(query)
            start = '<pad>'
            end = '<extra_id_0>'
            pattern_0 = s[s.find(start)+len(start):s.rfind(end)]
            pattern_list = []
            pattern_list.append(pattern_0)
            i = 0
            while i < total_chunk-1:
                if i == (total_chunk -2):
                    start = f'<extra_id_{sent_token_id}>'
                    end = '</s>'
                    a = s[s.find(start)+len(start):s.rfind(end)]
                    pattern_list.append(a) 
                    i +=1
                else:
                    start = f'<extra_id_{sent_token_id}>'
                    end = f'<extra_id_{sent_token_id+1}>'
                    a = s[s.find(start)+len(start):s.rfind(end)]
                    pattern_list.append(a)
                    sent_token_id += 1 
                    i +=1
            
            sent_token_id = 0
            for i in range(len(pattern_list)):
                a = f'<extra_id_{sent_token_id}>'
                text = text.replace(a, pattern_list[i].strip())
                sent_token_id += 1
            gen_list.append(text.strip())
            
            
        if first_token_mask is False and last_token_mask is False:
            sent_token_id = 0
            s = str(query)
            pattern_list = []
            while i < total_chunk:
                start = f'<extra_id_{sent_token_id}>'
                end = f'<extra_id_{sent_token_id+1}>'
                a = s[s.find(start)+len(start):s.rfind(end)]
                pattern_list.append(a)
                sent_token_id += 1 
                i +=1
                
            sent_token_id = 0
            for i in range(len(pattern_list)):
                a = f'<extra_id_{sent_token_id}>'
                text = text.replace(a, pattern_list[i].strip())
                sent_token_id += 1
            gen_list.append(text.strip())    
            
        if first_token_mask is False and last_token_mask is True:
            sent_token_id = 0
            s = str(query)
            pattern_list = []
            while i < total_chunk:
                if i == total_chunk -1:
                    start = f'<extra_id_{sent_token_id}>'
                    end = '</s>'
                    a = s[s.find(start)+len(start):s.rfind(end)]
                    pattern_list.append(a) 
                    i +=1
                else:
                    start = f'<extra_id_{sent_token_id}>'
                    end = f'<extra_id_{sent_token_id+1}>'
                    a = s[s.find(start)+len(start):s.rfind(end)]
                    pattern_list.append(a)
                    sent_token_id += 1 
                    i +=1 
                
            sent_token_id = 0
            for i in range(len(pattern_list)):
                a = f'<extra_id_{sent_token_id}>'
                text = text.replace(a, pattern_list[i].strip())
                sent_token_id += 1
            gen_list.append(text.strip())     
                

100%|███████████████████████████████████████████| 32/32 [01:33<00:00,  2.91s/it]


In [20]:
for i in range(len(sentences[0:10])):
    print(sentences[i])
    print(input_list[i])
    print(gen_list[i])

心筋症によるショックが遷延し免疫グロブリンが奏効した月経関連黄色ぶどう球菌性トキシックショック症候群の1例
心筋症による<extra_id_0>し免疫グロブリン<extra_id_1>黄色ぶどう球菌性トキシックショック<extra_id_2>例
心筋症による心筋症を合併し免疫グロブリン低下を伴う黄色ぶどう球菌性トキシックショック症の1例
Toxic shock syndrome(TSS)は黄色ぶどう球菌やA群 β 溶血性連鎖球菌が産生するスーパー抗原に起因した感染性疾患である.
Toxicshock<extra_id_0>ぶどう球菌やA群β溶血性<extra_id_1>産生するスーパー抗原<extra_id_2>
Toxicshocksyndromeはぶどう球菌やA群β溶血性連鎖球菌が産生するスーパー抗原の総称。</s
スーパー抗原によって循環不全や多臓器不全を呈し致命的な経過をとりうるが,心筋症を合併した報告は少ない.
<extra_id_0>よって<extra_id_1>多<extra_id_2>呈し致命的な経過をとりうるが,<extra_id_3>し<extra_id_4>少ない.
よってMeckel憩室は多臓器障害を呈し致命的な経過をとりうるが,肺塞栓症を合併し肺塞栓症を合併少ない.
今回我々は月経に関連した黄色ぶどう球菌性TSSに心筋症を合併し抗菌薬治療と免疫グロブリン投与で軽快した成人の1例を経験したため報告する.
今回我々は月経に関連<extra_id_0>ぶどう球菌<extra_id_1>心筋症を合併<extra_id_2>で軽快<extra_id_3>の<extra_id_4>経験した<extra_id_5>.
今回我々は月経に関連した黄色ぶどう球菌による黄色心筋症を合併したので軽快した月経の症例を経験したので報告する.
症例患者:39歳女性.主訴:発熱,頭痛.既往歴:特記すべき既往歴なし.現病歴:入院前日から発熱と頭痛があり当院外来を受診した.39.0度の発熱があるが,その他バイタルサインや身体所見,血液検査所見で特記なく対症療法で経過観察された.帰宅後も症状が続き,当院外来を再診した.
症例患者<extra_id_0>女性.主訴:発熱<extra_id_1>既往歴:特記<extra_id_2>病歴:入院<extra_id_3>と頭痛が<extr

In [8]:
sentences[0:10]

['心筋症によるショックが遷延し免疫グロブリンが奏効した月経関連黄色ぶどう球菌性トキシックショック症候群の1例',
 'Toxic shock syndrome(TSS)は黄色ぶどう球菌やA群 β 溶血性連鎖球菌が産生するスーパー抗原に起因した感染性疾患である.',
 'スーパー抗原によって循環不全や多臓器不全を呈し致命的な経過をとりうるが,心筋症を合併した報告は少ない.',
 '今回我々は月経に関連した黄色ぶどう球菌性TSSに心筋症を合併し抗菌薬治療と免疫グロブリン投与で軽快した成人の1例を経験したため報告する.',
 '症例患者:39歳女性.主訴:発熱,頭痛.既往歴:特記すべき既往歴なし.現病歴:入院前日から発熱と頭痛があり当院外来を受診した.39.0度の発熱があるが,その他バイタルサインや身体所見,血液検査所見で特記なく対症療法で経過観察された.帰宅後も症状が続き,当院外来を再診した.',
 '再診時体温40.0度,血圧100/70mmHg,脈拍数98/分(整),呼吸数16/分,SpO2 98%(室内気)で意識清明だった.',
 '上下肢や体幹部に明らかな紅斑なく,その他身体所見上特記すべき所見はなかった.',
 '血液検査所見は,好中球数10,820/μL,CRP14.4mg/dLと上昇し,クレアチニン1.17mg/dLと上昇を示した.',
 '尿中白血球1視野に100以上と上昇を示し,胸腹部CT検査上発熱源を示唆する有意な所見や両側腎盂拡張所見がなく,急性単純性腎盂腎炎として入院となった.',
 '入院後経過:尿培養と血液培養採取後cefmetazole(CMZ)1g×3/day投与を開始された.']

In [9]:
gen_list[0:10]

['心筋症によるショックが治療に奏効した非外傷性トキシックショック症候群の1例',
 'ヒトメタニューモウイルス(hMPV)は,Clostridiumbactrumbactrumbactrumbactrumbactrumbactrumbactrumbactrumbac群β溶血性連鎖球菌ューモウイルス(hMPV<extra_id_0>,Clostridiumbactrumbactrumbactrumbactrumbactrumbactrumbactrumbactrumbacに起因した感染性疾患である.',
 'スーパーインフルは循環不全や多臓器障害を伴う経過をとりうるが,心筋症を合併した報告は少ない.',
 '今回,COVID-19に関連した黄色ぶどう球菌性TSSに心筋症を合併し抗菌薬投与後に軽快した成人の症例を経験したため報告する.',
 '症例患者:39歳女性.主訴:発熱,頭痛.既往歴:特記事項なし.現病歴:入院前日から発熱と頭痛があり当院を受診した.39.発熱はあるが,その他バイタルサインや身体所見,既往歴はなく対症療法であった.帰宅後も症状が続き,当院を受診した.',
 '再診時体温40.0°C,血圧70mmHg,脈拍92回(整),呼吸数16分,SpO298%(室内気)で意識清明だった.',
 '上下肢に明らかな紅斑なく,その他身体所見上特記すべき異常所見なし。',
 '血液検査所見は,白血球数,L,CRP,クレアチニン値が上昇し,クレアチニン値もやや上昇を示した.',
 '尿中白血球1視野に100以上と上昇を示し,臨床症状上発熱源を伴う発熱源や両側腎盂拡張はなく,単純性腎盂拡張のみとなった.',
 '入院後経過:尿培養採取後にcefmetazole(CMZ)100mg/day投与を開始された.']

In [10]:
input_list[0:10]

['心筋症によるショック<extra_id_0>奏効した<extra_id_1>性トキシックショック症候群の1例',
 '<extra_id_0>)は<extra_id_1>群β溶血性連鎖球菌<extra_id_2>に起因した感染性疾患である.',
 'スーパー<extra_id_0>不全や多臓器<extra_id_1>経過をとりうるが,<extra_id_2>報告は少ない.',
 '<extra_id_0>関連した黄色ぶどう球菌性TSSに心筋症を合併し抗菌薬<extra_id_1>軽快した成人の<extra_id_2>たため報告する.',
 '症例患者:39歳女性.主訴:発熱,頭痛.既往歴<extra_id_0>なし.現病歴:入院前日から発熱と頭痛<extra_id_1>受診した.39.<extra_id_2>あるが,その他バイタルサインや身体所見,<extra_id_3>なく対症療法で<extra_id_4>帰宅後も症状が続き<extra_id_5>した.',
 '再診時体温40.0<extra_id_0>70mmHg,<extra_id_1>(整<extra_id_2>分,SpO298%(室内気)で意識清明だった.',
 '上下肢<extra_id_0>な紅斑なく,その他身体所見上特記すべき<extra_id_1>',
 '血液検査所見は,<extra_id_0>L,CRP<extra_id_1>上昇し,クレアチニン<extra_id_2>上昇を示した.',
 '尿中白血球1視野に100以上と上昇を示し<extra_id_0>上発熱源を<extra_id_1>や両側腎盂拡張<extra_id_2>単純性腎盂<extra_id_3>となった.',
 '入院後経過:尿培養<extra_id_0>cefmetazole(CMZ)<extra_id_1>day投与を開始された.']